## Notebook setup

In [2]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.api import widgets
from openbb_terminal.api import openbb
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
# predictions = True
# try:
#     openbb.stocks.pred.models
# except Exception as e:
#     predictions = False

# TODO Fix predictions virtual path on api refactored

predictions = False

### Header

In [3]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = widgets.html_report_stylesheet()
with open("OpenBB_reports_logo.png", "rb") as image_file:
    openbb_image_encoded = base64.b64encode(image_file.read())

In [15]:
author = "Diogo Sousa"
report_title = f"Global economy report"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_name = "economy_test"
report_title, report_date, report_time, report_timezone

('Global economy report',
 '25 August, 2022',
 '15:47',
 datetime.timezone(datetime.timedelta(seconds=3600), 'GMT Daylight Time'))

## Data

In [16]:
overview = openbb.economy.overview()
overview

,,Price,Chg,%Chg
0,DJIA,33131.03,161.80,0.49
1,S&P 500,4183.39,42.62,1.03
2,Nasdaq Composite,12600.61,169.08,1.36
3,Japan: Nikkei 225,28479.01,165.54,0.58
4,UK: FTSE 100,7492.45,20.94,0.28
5,Crude Oil Futures,94.66,-0.23,-0.24
6,Gold Futures,1770.10,8.60,0.49
7,Yen,136.66,-0.47,-0.34
8,Euro,0.9996,0.0030,0.30


In [17]:
news = openbb.common.news()
news

,title,link,published
0,Peloton Gives Gloomy Forecast in Sign Comeback...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 13:49:43 GMT"
1,"With Allied Mind (LON:ALM) Delisting From LSE,...",https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 07:45:00 GMT"
2,Huge Bearish Bets Against Hong Kong Stocks May...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 07:21:19 GMT"
3,Futures Rise as China Lifts Mood Ahead of Powe...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 13:57:18 GMT"
4,China's Cnooc More Than Doubles Profits on Hig...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 10:18:49 GMT"
...,...,...,...
95,The Fall - Bloomberg,https://www.bloomberg.com/news/audio/2022-08-2...,"Thu, 25 Aug 2022 10:42:53 GMT"
96,Sweden Orders Review Into Justification of Cor...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 09:35:15 GMT"
97,Fed Policy Revamp Turns Two in Jackson Hole. I...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 11:00:00 GMT"
98,India's NDTV Says Adani Needs Regulator’s Appr...,https://www.bloomberg.com/news/articles/2022-0...,"Thu, 25 Aug 2022 11:09:14 GMT"


## Render the report template to a file

In [22]:
body = ""

img = f'<img src="data:image/png;base64,{openbb_image_encoded.decode()}" alt="OpenBB" style="width:144px;">'
body += widgets.header(
    img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>GLOBAL ECONOMY REPORT:</b>",
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Overview",
    ]
)

htmlcode = widgets.row([widgets.h(3, "Market overview") + overview.to_html()])
htmlcode += widgets.row(widgets.h(3, "Top news") + [news[:6].to_html()])

body += widgets.add_tab("Overview", htmlcode)

body += widgets.tab_clickable_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)